In [ ]:
import hpq


def aggregate(iter):
    outright_first = False
    outright = None
    implied = None
    last = None

    def flush():
        nonlocal outright
        nonlocal implied
        nonlocal last
        if outright_first and outright is not None:
            yield outright
            outright = None
        if implied is not None:
            yield implied
            implied = None
        if outright is not None:
            yield outright
            outright = None
        last = None

    for obj in iter:
        if "transaction_id" not in obj.keys():
            for flushed in flush():
                yield flushed
            continue
        if last is None:
            last = obj
            continue
        if obj["transaction_id"] != last["transaction_id"]:
            for flushed in flush():
                yield flushed
        last = obj
        if obj["implied"]:
            implied = obj
            outright_first = False
            continue
        outright_first = True
        outright = obj
    for flushed in flush():
        yield flushed


def format(obj):
    del obj["type"]
    del obj["product_id"]
    del obj["name"]

    def flatten(key):
        inner = obj[key]
        del obj[key]
        if inner is not None:
            del inner["buy"]
            del inner["level"]
            for k in inner.keys():
                actual = inner[k]
                if k == "exchange_timestamp" or k == "receipt_timestamp":
                    actual = hpq.format_timestamp(actual)
                obj[key + "_" + k] = actual

    if "bid" in obj.keys():
        flatten("bid")
    if "ask" in obj.keys():
        flatten("ask")
    return hpq.format(obj)

In [ ]:
import hpq
import ijson
import IPython.display
import tabulate

connection = hpq.create_web_socket_client()
request = {
    "query": "topofbook",
    "source": "cme",
    "product": "CLZ2",
    "date": "20221107",
    "start_time": "08:04:56",
    "end_time": "08:04:56.4",
    "timestamp_mode": "exchange",
}
rows = list(filter(format, aggregate(ijson.items(connection.stream(request), "item"))))
IPython.display.display(IPython.display.HTML("<h1>Top of Book Changes</h1>"))
IPython.display.display(IPython.display.HTML("<h2>Aggregated on Transaction</h2>"))
count = len(rows)
IPython.display.display(IPython.display.HTML(f"<p>{count} results</p>"))
table = tabulate.tabulate(rows, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))
rows = list(filter(format, ijson.items(connection.stream(request), "item")))
IPython.display.display(IPython.display.HTML("<h2>All Changes</h2>"))
count = len(rows)
IPython.display.display(IPython.display.HTML(f"<p>{count} results</p>"))
table = tabulate.tabulate(rows, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))